# Getting started
ExplainabilityMethods.jl can be used on any classifier.
In this first example, we will look at attributions on a LeNet5 model that was pretrained on MNIST.

### Loading the model

In [1]:
using ExplainabilityMethods
using Flux
using BSON

model = BSON.load("../model.bson", @__MODULE__)[:model]

Chain(
  Conv((5, 5), 1 => 6, relu),           # 156 parameters
  MaxPool((2, 2)),
  Conv((5, 5), 6 => 16, relu),          # 2_416 parameters
  MaxPool((2, 2)),
  Flux.flatten,
  Dense(256, 120, relu),                # 30_840 parameters
  Dense(120, 84, relu),                 # 10_164 parameters
  Dense(84, 10),                        # 850 parameters
)                   # Total: 10 arrays, 44_426 parameters, 174.867 KiB.

### Loading MNIST
We use MLDatasets to load a single image from the MNIST dataset:

In [2]:
using MLDatasets
using ImageCore

index = 10
x, y = MNIST.testdata(Float32, index)

MNIST.convert2image(x)

By convention in Flux.jl, this input needs to be resized to WHCN format by adding a color channel and batch dimensions.

In [3]:
input = reshape(x, 28, 28, 1, :);

## Calling the analyzer
We can now select an analyzer of our choice
and call `analyze` to get an `Explanation`:

In [4]:
analyzer = LRPZero(model)
expl = analyze(input, analyzer);

This `Explanation` bundles the following data:
* `expl.attribution`: the analyzer's attribution
* `expl.output`: the model output
* `expl.neuron_selection`: the neuron index of used for the attribution
* `expl.analyzer`: a symbol corresponding the used analyzer, e.g. `:LRP`

Finally, we can visualize the `Explanation` through heatmapping:

In [5]:
heatmap(expl)

Or get the same result by combining both analysis and heatmapping into one step:

In [6]:
heatmap(input, analyzer)

## Neuron selection
By passing an additional index to our call to `analyze`, we can compute the attribution
with respect to a specific output neuron.
Let's see why the output wasn't interpreted as a 4 (output neuron at index 5)

In [7]:
heatmap(input, analyzer, 5)

This heatmap shows us that the "upper loop" of the hand-drawn 9 has negative relevance
with respect to the output neuron corresponding to digit 4!

## Automatic heatmap presets
Currently, the following analyzers are implemented:

```
├── Gradient
├── InputTimesGradient
└── LRP
    ├── LRPZero
    ├── LRPEpsilon
    └── LRPGamma
```

Let's try `InputTimesGradient`

In [8]:
analyzer = InputTimesGradient(model)
heatmap(input, analyzer)

and `Gradient`

In [9]:
analyzer = Gradient(model)
heatmap(input, analyzer)

As you can see, the function `heatmap` automatically applies common presets for each method.

Since `InputTimesGradient` and LRP both compute attributions, their presets are similar.
Gradient methods however are typically shown in grayscale.

## Custom heatmap settings
We can partially or fully override presets by passing keyword arguments to `heatmap`:

In [10]:
using ColorSchemes
heatmap(expl; cs=ColorSchemes.jet)

In [11]:
heatmap(expl; reduce=:sum, normalize=:extrema, cs=ColorSchemes.inferno)

For the full list of keyword arguments, refer to the `heatmap` documentation.

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*